# RunnableParallel

#### 필수 라이브러리 다운로드

In [79]:
!pip install --upgrade --quiet  langchain-core langchain-community langchain-openai

#### openai api key 등록

In [80]:
import os
os.environ["OPENAI_API_KEY"]      = "sk-*******************************************"

In [93]:
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

#vector db에 text 데이터 수집
vectorstore = FAISS.from_texts(
    ["harrison worked at kensho"], embedding=OpenAIEmbeddings()
)

#vector db retriever 생성
retriever = vectorstore.as_retriever()

#prompt template 생성
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

#모델 생성
model = ChatOpenAI()

#LCEL retrieval chain 생성
retrieval_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

#Chain test
retrieval_chain.invoke("where did harrison work?")

'Harrison worked at Kensho.'

In [96]:
from langchain_core.runnables import RunnableParallel

#병렬 Runnable을 적용한 chain 생성
#json input 적용
retrieval_chain2 = (
    RunnableParallel({"context": retriever, "question": RunnablePassthrough()})
    | prompt
    | model
    | StrOutputParser()
)

#체인 테스트
retrieval_chain2.invoke("where did harrison work?")

'Harrison worked at Kensho.'

In [97]:
from langchain_core.runnables import RunnableParallel

#병렬 Runnable을 적용한 chain 생성
#함수 내 변수 적용
retrieval_chain3 = (
    RunnableParallel(context=retriever, question=RunnablePassthrough())
    | prompt
    | model
    | StrOutputParser()
)

retrieval_chain3.invoke("where did harrison work?")

'Harrison worked at Kensho.'

## Parallelize steps

In [98]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel
from langchain_openai import ChatOpenAI

#모델 생성
model = ChatOpenAI()

# 멀티 체인 생성
joke_chain = ChatPromptTemplate.from_template("tell me a joke about {topic}") | model
poem_chain = (
    ChatPromptTemplate.from_template("write a 2-line poem about {topic}") | model
)

# 병렬 체인 생성
map_chain = RunnableParallel(joke=joke_chain, poem=poem_chain)

# 테스트
map_chain.invoke({"topic": "bear"})

{'joke': AIMessage(content='Why did the bear bring a flashlight to the party? \n\nBecause he heard it was going to be a "beary" bright affair!', response_metadata={'finish_reason': 'stop', 'logprobs': None}),
 'poem': AIMessage(content='In the quiet forest, the bear roams free,\nMajestic and strong, a sight to see.', response_metadata={'finish_reason': 'stop', 'logprobs': None})}

In [99]:
%%timeit

#joke chain 테스트
joke_chain.invoke({"topic": "bear"})

896 ms ± 180 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [100]:
%%timeit

#poem chain test
poem_chain.invoke({"topic": "bear"})

727 ms ± 146 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [101]:
%%timeit

#multi chain test
map_chain.invoke({"topic": "bear"})

889 ms ± 201 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
